In [ ]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data=pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test_data=pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

In [ ]:
train_data.shape,test_data.shape

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.nunique()

In [ ]:
type(train_data.iloc[0,0])

In [ ]:
train_data['Dates']=pd.to_datetime(train_data['Dates'])
test_data['Dates']=pd.to_datetime(test_data['Dates'])

In [ ]:
train_data['month']=[x.month for x in train_data['Dates']]
test_data['month']=[x.month for x in test_data['Dates']]

In [ ]:
train_data['year']=[x.year for x in train_data['Dates']]
test_data['year']=[x.year for x in test_data['Dates']]

In [ ]:
train_data.drop(['Dates','Descript','Resolution'],axis=1,inplace=True)
test_data.drop(['Dates'],axis=1,inplace=True)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.isnull().value_counts()

In [ ]:
train_data.head()

In [ ]:
train_data.nunique()

In [ ]:
train_data.drop('Address',inplace=True,axis=1)
test_data.drop('Address',inplace=True,axis=1)

In [ ]:
Y=train_data['Category']
train_data.drop('Category',inplace=True,axis=1)

In [ ]:
train_data['X']=round(train_data['X'],3)
test_data['X']=round(test_data['X'],3)
train_data['Y']=round(train_data['Y'],3)
test_data['Y']=round(test_data['Y'],3)

In [ ]:
for i in test_data.columns[1:]:
    test_data=pd.concat([test_data,pd.get_dummies(test_data[i])],axis=1)
    test_data.drop(i,axis=1,inplace=True)

In [ ]:
for i in train_data.columns:
    train_data=pd.concat([train_data,pd.get_dummies(train_data[i])],axis=1)
    train_data.drop(i,axis=1,inplace=True)

In [ ]:
test_data.shape,train_data.shape

In [ ]:
test_data.head()

In [ ]:
ids=test_data["Id"]
test_data=test_data[train_data.columns]
test_data.insert(0,'Id',ids)

In [ ]:
Y=pd.get_dummies(Y)

In [ ]:
cols=Y.columns

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train_data,Y,train_size=0.5,test_size=0.2, random_state=41)

In [ ]:
from keras.layers import Dense,Dropout
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Dense(128,input_shape=(X_train.shape[1],)))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(39,activation='softmax'))
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train=model.fit(x=X_train,y=y_train,batch_size=1000,epochs=10,verbose=2,validation_data=(X_test,y_test))

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(train.history['accuracy'],'r',label='Training accuracy')
plt.plot(train.history['val_accuracy'],'b',label='Validation accuracy')
plt.legend()

In [ ]:
test_data.head()

In [ ]:
test_data.head()
pred_final=model.predict(test_data.drop('Id',axis=1))
pred_final

In [ ]:
submission = pd.DataFrame(data=pred_final,columns=y_train.columns)
submission.insert(0,'Id',ids)
submission.to_csv('../working/submission.csv', index=False)


In [ ]:
submission.head()

In [ ]:
submission.info()